In [1]:
import src.modelling_fef2575.hfef2575 as hfef2575
import src.modelling_fev1.pred_fev1 as hfev1
import src.models.helpers as mh
import plotly.graph_objects as go

# Compare prior for HFEV1 and HFEF2575

In [2]:
HFEV1 = mh.VariableNode("Healthy FEV1 (L)", 1, 8, 0.05, prior=None)

for age in [20, 60]:
# for age in [30]:
    for sex in ["Male"]:
    # for sex in ['Female', 'Male']:
        for height in [180]:
        # for height in [160, 170, 180, 190]:

            p_fef = hfef2575.calc_hfef2575_prior(HFEV1.bins, height, age, sex)
            p_fev1 = hfev1.calc_hfev1_prior(HFEV1.bins, height, age, sex)

            lms_fef = hfef2575.calc_predicted_value_LMS_straight(height, age, sex)
            print(
                f'HFEF2575 std up: {lms_fef["sigma_up"]:.2f}, std down: {lms_fef["sigma_down"]:.2f}'
            )
            lms_fev1 = hfev1.calc_predicted_value_LMS_straight(height, age, sex)
            print(
                f'HFEV1 std up: {lms_fev1["sigma_up"]:.2f}, std down: {lms_fev1["sigma_down"]:.2f}'
            )

            fig = go.Figure()
            fig.add_trace(
                go.Scatter(
                    x=HFEV1.bins, y=p_fev1, mode="lines+markers", name="HFEV1 prior"
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=HFEV1.bins, y=p_fef, mode="lines+markers", name="HFEF2575 prior"
                )
            )
            fig.update_layout(title=f"{height}cm, {age}yr, {sex}")
            fig.show()

HFEF2575 std up: 1.27, std down: 1.13
HFEV1 std up: 0.54, std down: 0.55


HFEF2575 std up: 1.22, std down: 1.01
HFEV1 std up: 0.53, std down: 0.54


# FEF25-75

In [2]:
# pred FEF25-57 with age and height
import pandas as pd
import src.modelling_fev1.gli_models_helpers as glih
from plotly.subplots import make_subplots

HFEV1 = mh.VariableNode("Healthy FEV1 (L)", 1, 8, 0.05, prior=None)

sex = "Female"
height = 165
df_age = pd.DataFrame({"Age": range(3, 60)})
x = df_age.apply(
    lambda x: glih.calc_predicted_value_LMS(
        hfef2575.load_FEF2575_LMS_spline_vals(x.Age, sex),
        hfef2575.load_FEF2575_LMS_coeffs(sex),
        height,
        x.Age,
    ),
    axis=1,
)
# Unpack x to get m sigma up sigma down
df_age["M female"] = x.apply(lambda x: x["M"])
df_age["sigma_up Female"] = x.apply(lambda x: x["sigma_up"])
sex = "Male"
x = df_age.apply(
    lambda x: glih.calc_predicted_value_LMS(
        hfef2575.load_FEF2575_LMS_spline_vals(x.Age, sex),
        hfef2575.load_FEF2575_LMS_coeffs(sex),
        height,
        x.Age,
    ),
    axis=1,
)
df_age["M male"] = x.apply(lambda x: x["M"])
df_age["sigma_up Male"] = x.apply(lambda x: x["sigma_up"])


age = 25
sex = "Female"
df_height = pd.DataFrame({"Height": range(140, 190)})
x = df_height.apply(
    lambda x: glih.calc_predicted_value_LMS(
        hfef2575.load_FEF2575_LMS_spline_vals(age, sex),
        hfef2575.load_FEF2575_LMS_coeffs(sex),
        x.Height,
        age,
    ),
    axis=1,
)
df_height["M female"] = x.apply(lambda x: x["M"])
df_height["sigma_up Female"] = x.apply(lambda x: x["sigma_up"])
sex = "Male"
x = df_height.apply(
    lambda x: glih.calc_predicted_value_LMS(
        hfef2575.load_FEF2575_LMS_spline_vals(age, sex),
        hfef2575.load_FEF2575_LMS_coeffs(sex),
        x.Height,
        age,
    ),
    axis=1,
)
df_height["M male"] = x.apply(lambda x: x["M"])
df_height["sigma_up Male"] = x.apply(lambda x: x["sigma_up"])

In [5]:
# Evolution with age, height
fig = make_subplots(rows=1, cols=2, shared_yaxes=False)
fig.add_trace(
    go.Scatter(
        x=df_age.Age,
        y=df_age['M male'],
        mode="markers+lines",
        name="Age (male)",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=df_age.Age,
        y=df_age['M female'],
        mode="markers+lines",
        name="Age (female)",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=df_height.Height,
        y=df_height["M male"],
        mode="markers+lines",
        name="Height (male)",
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=df_height.Height,
        y=df_height["M female"],
        mode="markers+lines",
        name="Height (female)",
    ),
    row=1, col=2
)
fig.update_yaxes(title_text="FEF25-75 (L/s)", row=1, col=1)
# Add figure title: Sensitivity of std dev to age and height
fig.update_layout(title_text="Evolution of FEF25-75 with age, height")
# Update xaxis properties
fig.update_xaxes(title_text="Age (years), height fixed to 165cm", row=1, col=1)
fig.update_xaxes(title_text="Height (cm), age fixed to 25yr", row=1, col=2)
fig.show()

In [6]:

# Plot df_age sigma up with age
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig.add_trace(
    go.Scatter(
        x=df_age.Age,
        y=df_age["sigma_up Female"],
        mode="markers+lines",
        name="Age (female)",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=df_age.Age,
        y=df_age["sigma_up Male"],
        mode="markers+lines",
        name="Age (male)",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=df_height.Height,
        y=df_height["sigma_up Female"],
        mode="markers+lines",
        name="Height (female)",
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=df_height.Height,
        y=df_height["sigma_up Male"],
        mode="markers+lines",
        name="Height (male)",
    ),
    row=1, col=2
)
fig.update_yaxes(title_text="Deviation at Z-Score=1", row=1, col=1)
# Add figure title: Sensitivity of std dev to age and height
fig.update_layout(title_text="Sensitivity of the deviation from mean to age, height and sex")
# Update xaxis properties
fig.update_xaxes(title_text="Age (years), height fixed to 165cm", row=1, col=1)
fig.update_xaxes(title_text="Height (cm), age fixed to 25yr", row=1, col=2)
fig.show()